In [ ]:
!pip install moviepy

In [ ]:
pip install SpeechRecognition pydub


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
import moviepy.editor as mp

clip = mp.VideoFileClip("/content/5 Minutes for the Next 50 Years - Mathhew McConaughey Motivational Speech.mp4")
clip.audio.write_audiofile("my_audio.mp3")

  if event.key is 'enter':



MoviePy - Writing audio in my_audio.mp3


MoviePy - Done.


In [10]:
import math
from pydub import AudioSegment
import speech_recognition as sr

# Load audio file
sound = AudioSegment.from_file("/content/converted_audio.wav")
chunk_length_ms = 60000  # 60 seconds
chunks = math.ceil(len(sound) / chunk_length_ms)

r = sr.Recognizer()
full_text = []

for i in range(chunks):
    start = i * chunk_length_ms
    end = min((i + 1) * chunk_length_ms, len(sound))
    chunk = sound[start:end]
    chunk_name = f"chunk_{i}.wav"
    chunk.export(chunk_name, format="wav")

    with sr.AudioFile(chunk_name) as source:
        audio_data = r.record(source)
        try:
            text = r.recognize_google(audio_data)
            full_text.append(text)
        except sr.UnknownValueError:
            print(f"Chunk {i}: Audio not understood")
        except sr.RequestError as e:
            print(f"Chunk {i}: API request failed: {e}")
final_text = " ".join(full_text)
print(final_text)


I'm going to talk to you about some things I've learned in my journey most from experience some of them I heard and passing many of them I'm still practicing but all of them I do believe are true life is not easy it is not don't try to make it that way life's not fair and never was it is it now and it won't ever be do not fall into the Trap the entitlement trap a feeling like you're a victim you are not get over it and get on with it so the question to ask yourself that question are you answering me change over time and that's fine but do yourself whatever prioritize who you are who you want to be and don't spend time with anything that antagonizes your character be brave take the hill but first answer that question what's my hill but first we have to define success for sale play I know who I am not process of elimination defining ourselves by what we are not is the first step that leads us to really knowing who we are you know that group of friends that you hang out with that it reall

In [11]:
print(final_text)

I'm going to talk to you about some things I've learned in my journey most from experience some of them I heard and passing many of them I'm still practicing but all of them I do believe are true life is not easy it is not don't try to make it that way life's not fair and never was it is it now and it won't ever be do not fall into the Trap the entitlement trap a feeling like you're a victim you are not get over it and get on with it so the question to ask yourself that question are you answering me change over time and that's fine but do yourself whatever prioritize who you are who you want to be and don't spend time with anything that antagonizes your character be brave take the hill but first answer that question what's my hill but first we have to define success for sale play I know who I am not process of elimination defining ourselves by what we are not is the first step that leads us to really knowing who we are you know that group of friends that you hang out with that it reall

## RAG

In [12]:
#Splitting the document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = splitter.split_text(final_text)

In [16]:
#Converting texts into embeddings
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts, show_progress_bar=True)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
def search_query(query, k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    results = [texts[i] for i in indices[0]]
    return results


In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
gen_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def answer_query(query):
    context_chunks = search_query(query)
    context = "\n\n".join(context_chunks)

    prompt = f"""Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"""

    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = gen_model.generate(**inputs, max_new_tokens=256)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [23]:
question = "what is not fair?"
print(answer_query(question))


life
